### Оъявдение того, что будет происходить

Acknowledgement: this notebook is inspired by speech course from Andrei Malinin

In [ ]:
# %pip install -r ../requiremets.txt

In [ ]:
import os
import requests
from pathlib import Path
from urllib.parse import urlencode

from IPython.display import display, Audio

In [ ]:
def download_file(public_link):
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    final_url = base_url + urlencode(dict(public_key=public_link))
    response = requests.get(final_url)
    parse_href = response.json()['href']

    url = parse_href
    start_filename = url.find('filename=')
    end_filename = url[start_filename:].find('&')
    end_name = start_filename + end_filename
    filename = url[start_filename:end_name][9:]
    download_url = requests.get(url)
    final_link = os.path.join(os.getcwd(), filename)
    with open(final_link, 'wb') as ff:
        ff.write(download_url.content)


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from plotting_utils import *
from transforms import *
from tests import *

In [ ]:
### To download the file uncomment the following line

# link_to_archive = "https://disk.yandex.ru/d/aEqBwRPwpIgJUQ"
# download_file(link_to_archive)
# !unzip 01_DSP.zip
# !rm 01_DSP.zip
# !mkdir -p ../data
# !mv 01_DSP ../data

In [ ]:
sample_rate = 22050
data_path = Path("../data")
seminar_name = "01_DSP"
samples_path = data_path / seminar_name / "examples"

In [ ]:
def create_examples_dict(directory="../data/01_DSP/examples", sample_rate=sample_rate):
    directory = Path(directory)

    examples = {}
    for wav_path in directory.iterdir():
        if wav_path.suffix == ".wav":
            wav, sr = librosa.load(str(wav_path))
            assert sr == sample_rate, f"Working only with audio of sample_rate {sample_rate}"
            examples[wav_path.stem] = wav
    return examples

examples = create_examples_dict(directory=samples_path)
wav = examples["babenko"]

We will apply our transformation to this wav. Then see how it changes.

### 1. WAV (Waveform Audio Format)

Sound is recorded by a microphone. It registers a serie of amplitudes of air-pressure. The frequency, at which microphone records amplitudes is called sample-rate. Typicaly, this frequency can be 16kHz, 22kHz or 48kHz, depending on what quality of audio one aim to record.


In [ ]:
wav_44kHz, orig_sr = librosa.load(str(data_path / seminar_name / "sample_44kHz.wav"), sr=44100)
print(f"Original sample rate: {orig_sr}")
for target_sr in [44100, 24000, 22050, 16000, 8000, 4000]:
    resampled_wav = librosa.resample(wav_44kHz, orig_sr=orig_sr, target_sr=target_sr)
    print(f"Resampled SR: {target_sr}")
    display(Audio(resampled_wav, rate=target_sr))

In this seminar we will use samples recorded at 22kHz, which means that each second of recorded sound will contain 22 thousand (22050 actually) amplitude samples. Consider the short audio files below:

In [ ]:
print(f"{wav.shape=} {sample_rate=}")
display(Audio(wav, rate=sample_rate))

In [ ]:
plot_wav(wav, sample_rate, end=None)

As you can see, waveform is a redundant representation. __4-second__ audio sample where only __7 words__ are said, contains __87040__ amplitude samples! This is extremely difficult to process directly, because there's so many of them! It is far more than a typical sequence length for NLP applications. Thus, we need to compress this information to a more manageable size!

A typical solution is to use __spectrogram__: instead of saving thousands of amplitudes, we can perform [Fourier transformation](https://en.wikipedia.org/wiki/Fourier_transform) to find which periodics are prevalent at each point in time. More formally, a spectrogram applies [Short-Time Fourier Transform (STFT)](https://en.wikipedia.org/wiki/Short-time_Fourier_transform) to small overlapping windows of the amplitude time-series. Let us describe this pipeline and step by step

### 1.1 Let's listen to different sample rates

### 2. Windowing
Our speech is non-stationary and varies over time. Thus, we do not want to apply DFT to the whole recording, as it does not help us to retrieve information from the recording. What we actually want - is to see how local spectral characteristics change over time. That's why we slice our recording into overlapping windows and apply DFT to this window. This slicing has two important characteristics:
- **Window_size** - the size of the window. Typically, it is 1024 or 2048 samples, which is roughly 0.05 - 0.1 seconds of audio. (Really so small)
- **Hop_length** - the size of the step, between starts of two contiguous windows. Usually is equals to window_size // 4.

As the hop_length is smaller than window size, the windows overlap on each other. And each sample from the wav is duplicated (window_size // hop_length) times.

Now you need to implement Windowing class.
- Middle of $i-th$ window (element with index $(win\_size - 1) // 2$) should lean on $(i \times hop\_length)-th$ of the original wav
- To do that zero-pad left and right borders of the wav
- The expectend number of windows is `(wav_size - window_size % 2) // hop_length + 1`

In [ ]:
# Assignment: implement transforms.Windowing

In [ ]:
assert test_windowing(testing_class=Windowing, n_repeats=1000)

In [ ]:
start, length = 4000, 3584
plot_windowing(wav[start:start + length], windowing_class=Windowing, n_subpictures=4)

This picture shows, how $0.16$ seconds od audio turn into several short frames, $0.09$ seconds each. The important thing here is that they intersect, quite significantly.

Let's apply apply the transformation to the whole wav, and see what we've got.

In [ ]:
windowing_transform = Windowing(window_size=2048, hop_length=512)
windows = windowing_transform(wav)
plot_windowed_wav(windows)


Doesn't look like something meaningful, so let's continue doing some other transforms

### 3. Hann window
If we apply DFT to each window, resulted from the previous step, we'll get so called spectral leakage. Let me try to give you an intuition, of what it is:

Fourier transform assumes, that the function is periodic and continuous.
We can make function periodic, by pretending, that we loop it.
But when we loop it, we break its continuousness, as the function does not match at the ends.
To address this, we need our function to be equal from both sides, for example make it equal to zero.
To smoothly bring the function to zero at the ends, we multiply it by hahn function elementwise.
This function equals 1 in the middle, and zero at the ends (like on following picture)

This guy explains that better:
https://www.youtube.com/shorts/xoE2rnwFROs

Weights for Hann window can be taken [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.windows.hann.html). 

In [ ]:
# Assignment: implement transforms.Hann

In [ ]:
assert test_hann(testing_class=Hann)

Let's see what we've got

In [ ]:
start1, start2, length = 4500, 10450, 2048 
demo_tensor = get_demo_tensor([wav[start1:start1 + length], wav[start2:start2 + length]])
plot_hann(demo_tensor)

Let's apply our new transform to the windows we've got from the previous step.

In [ ]:
hann_transform = Hann(window_size=2048)
hann_windows = hann_transform(windows)
plot_windowed_wav(hann_windows)

### 4. FFT

The Short-Time Fourier Transform (STFT) is based on the Fast Fourier Transform (FFT), which is a fast implementation of the Discrete Fourier Transform (DFT). DFT is actually what we want to do :)

In numpy, there are two types of DFTs: [np.fft.fft](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft.html#numpy.fft.fft) and [np.fft.rfft](https://numpy.org/doc/stable/reference/generated/numpy.fft.rfft.html#numpy.fft.rfft). Let's explore the differences between them.



It's important to note that the results of these functions are complex-valued arrays.

Below is a plot depicting the magnitudes and phases of both FFT and RFFT results.

In [ ]:
plot_fft_difference()

Let's understand, what happens on this plot.

We've seen a formulae of DFT on the lecture:
$$
X[k] = \sum_{n=0}^{N-1} x[n] e^{-2 \pi i \frac {k}{N}n}
$$

In this formulae $\frac{k}{N}$ is the frequency, that we have over the x-axis.
Let's deduce two important properties of this formulae.

1) For complex $x(k)$
$$
X[N + k] = \sum_{n=0}^{N-1} x[n] \exp(-2 \pi i \frac{N + k}{N} n) = \sum_{n=0}^{N-1} x[n] \exp(-2 \pi i \frac{k}{N} n - \cancel{2 \pi i n}) = \sum_{n=0}^{N-1} x[n] \exp(-2 \pi i \frac{k}{N} n) \\
X[N + k] = X[k]
$$

2) For real $x(k)$
$$
X[N - k] = \sum_{n=0}^{N-1} x[n] \exp(-2 \pi i \frac{N - k}{N} n) = \sum_{n=0}^{N-1} x[n] \exp(2 \pi i \frac{k}{N} n - \cancel{2 \pi i n}) = \sum_{n=0}^{N-1} x[n] \exp(2 \pi i \frac{k}{N} n) \\
X[N - k] = X^{*}[k]
$$

What are the conclusions from this formulae.
- The result of the formulae is periodical with period $N$. That's why if we want to calculate several values of this function for several frequencies, we can take frequencies $n$ from a range of size $N$.
- In the implementation of DFT in numpy ([np.fft.fft](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft.html#numpy.fft.fft])) this range is chosen to be $n \in [- \frac{N}{2}; + \frac{N}{2}]$, or in terms of numpy : $\frac{n}{N} \in [-\frac{1}{2}; +\frac{1}{2}]$.
- We work with real-value samples as input of this function. And we want the absolutes of the results. That means, that according to formulae 2, we will get a symmetric result, $|X(n)| == |X^{*}(N - n)|$.
- For this case we have another function [np.fft.rfft](https://numpy.org/doc/stable/reference/generated/numpy.fft.rfft.html), which calculates the DFT for range $n \in [0, \frac{N}{2}]$.

The same exact explanation you can see in [np.fft documentation](https://numpy.org/doc/stable/reference/routines.fft.html).

Now, given this knowledge, let's implement the DFT method.
What is it expected to do:
- Get the real-value matrix and apply DFT transformation to each of this column. 
- Return the real-value result.
- It has `n_freqs` parameter. If it is `None`, we return all of the frequencies. If it is an integer, we return only the `n_freqs` lowest frequencies. 

In [ ]:
# Implement transforms.DFT

In [ ]:
assert test_dft()

Now let's see what we've got 

In [ ]:
dft_transform = DFT()
spec = dft_transform(hann_windows)
plot_dft(spec)

### 4.1 Let's one more time understand, what has just happened

In [ ]:
def signal_fn(t: float):
    res = 1
    res = res + 1 * np.sin(2 * np.pi * 2 * t + 0.25 * np.pi) # + 0.25 * np.pi
    res = res + 2 * np.sin(2 * np.pi * 6 * t)
    res = res + 0.5 * np.sin(2 * np.pi * 14 * t - 0.5 * np.pi)
    return res

Let's say, we have a continuous signal, like this:

$$
x(t) = 1 + 1 \sin(2 \pi \cdot 2 t + \frac{\pi}{4}) + \sin(2 \pi \cdot 4 t) + 0.5 \sin(2 \pi \cdot 7 t - \frac{\pi}{2}) \ , \ t \in [0, +inf)
$$

Our sample rate define the time between two nearest samples $T_s = \frac{1s}{SR} \ $, let's apply $t \to n T_s $ 

$$
    x[n] = 1 + 3 \sin(2 \pi T_s \cdot n) + \sin(2 \pi T_s \cdot 4 n) + 0.5 \sin(2 \pi T_s \cdot 7 n) \ , \ n \in [0, +inf)
$$

Then we have taken only $N$ (`window_size`) samples from this sequence. And apply a DFT transformation to it, according to the formula:

$$
X[k] = \sum_{n=0}^{N-1} x[n] e^{-2 \pi i \frac {k}{N}n}
$$
According to this formula we get an array of $N$ values in time domain, and get an array of $N$ in frequency domain.


In [ ]:
plot_signal_fft(signal_fn, sample_rate=128, window_size=64)

**Conclusions**

- Sample rate defines the Nyquist frequency. It is the maximum frequency, we can estimate with fft.
- Then we choose the `window_size`, it defines the grid, at which we estimate our signal.

### 5. Energy/Intensity/Magnitude Spectrum

On the lecture, we have discussed, that for several applications we can be interested in energy, which is transmitted by given frequencies of the signal. For this purpose we can square the spectrogram.

To get the loudness, we can also take the logarithm. 

In [ ]:
plot_dft(np.log(spec ** 2))

But we want our transformation to be inverse, so we're not going to do that

### 6. MelScale

Initially one second of audio wav represented by 16k samples per second.
We have transformed our wav in more convenient form but bitrate of the wav stayed almost the same $~(sample\_rate \ // \ hop\_length) \times n\_fft \simeq 16k$ floats, representing 1 secong of audio. We won 2x space we get rid imaginary part of the signal and switched to the energy of harmonics. We lost space because overlaping of fft windows. 

But now we can use our knowledge of psychoacoustic as not all frequencies are useful.
Different frequencies of the spectrum have different contribution for audio perception.
Low frequencies are more important and high frequencies are less important as it was explained in the first lecture.
We can apply to spectrogram by multiplying amplitudes on mel-scale matrix.
You can see one below.

In [ ]:
plot_mel_scale()

By multiblying each column of the spectrogram on this matrix, we reduce the number of "frequencies".

As you can see low frequencies here have higher weight, higher frequencies - lower weights.
Very high frequencies are even ignored.

Another good property, that we have achieved - is that the importance of each mel bin for human comprehesion is equal.

We can also condider this transformation reversible.
For restoration of a spectrogram from mel, we can use pseudo-inverse matrix.

To get the mel matrix use [this](https://librosa.org/doc/latest/generated/librosa.filters.mel.html) function with hyperparameters `fmin=1, fmax=8192`

In [ ]:
# Implement transforms.Mel (both __call__ and restore methods)

In [ ]:
test_mel(testing_class=Mel)

In [ ]:
wav_to_spec = Wav2Spectrogram(window_size=1024, hop_length=256, n_freqs=None)
spec_to_mel = Mel(n_fft=1024, n_mels=80, sample_rate=22050)

spec = wav_to_spec(examples["diesel"])
mel = spec_to_mel(spec)

In [ ]:
plot_spec_mel(spec, mel)

Let's see how pseudo-inverse matrix restores the spectrogram

In [ ]:
restored_spec = spec_to_mel.restore(mel)
plot_rerstored_spec(spec, restored_spec)

Visually, we got pretty much the same. Yet the difference is still noticable.

Now we got the final representation of audio, which can then be fed to a neural network, or some other algorithm.
Why we like log-mel representation?

- It is more compact than spectrogam
- It is more informative than spectrogram
- You can watch on the spectrogram and notice the basic characteristics of the audio: pitch, amount of noise, loudness and so on.

What is constraints about logmel representation?

- There is no phase information which important for some noise reduction algorithms
- It is lossy comppression based on specific assumtion. Thus, it is not suitable for some specific scenarios like analysis of dolphins communications.

For analysis of human perceptive signals it works very well. So Let's draw some specs and try to notice some insights from them

In [ ]:
plot_wav_and_mel(examples)

### 7. Vocoding

In the upcoming chapters, we will need to convert the spectrogram back to the waveform, a process known as vocoding. Logically, this involves performing the inverse discrete Fourier transform of the Discrete Fourier Transform (DFT) results. However, in many cases, we work with log-mel spectrograms of a signal and lack the phase information of the signal.


The simplest way to address vocoding is GriffinLim algorithm.
Shortly, uses intersection of windows to windows to estimate phases.
It is implemented in Wav2Spectrogram.restore method for you.
We do not cover this algorithm in our course, but if you are curious enough to figure out how it works, follow the [link](https://paperswithcode.com/method/griffin-lim-algorithm)

### 8. Spectrogram transformations

Now, as we know how to transform wav to mel spectrogram and backward, we can implement some transformation of audio in mel space.

__Assignment:__

Go to the `transforms` file and mplement the following transformations over mels:

* `PitchDown`, `PitchUp` - make audio sound lower and higher.
* `SpeedUpDown` - increase/decrease speed of the recording by a given factor. (The same thing you do with this seminar if you watch it recorded=))
* `Loudness` - increase/decrease loudness by a given factor
* `TimeReverse` - inverse time
* `VerticalSwap` - let's see what will happen
* `WeakFrequenciesRemoval` - let's assume, that frequencies with low amplitude are not interesting for us. Let's zero frequencies smaller, than a certain quantile and see, what it changes.
* `Cringe1`, `Cringe2` - use your imagination to create a some funny transfomration

In [ ]:
def show_transformation(transform_class, wav, kwargs):
    sample_rate = 22050
    wav2mel = Wav2Mel(
        window_size=1024,
        hop_length=256,
        n_freqs=None,
        n_mels=80,
        sample_rate=sample_rate)

    mel_transform = transform_class(**kwargs)

    mel = wav2mel(wav)
    transformed_mel = mel_transform(mel)
    transformed_wav = wav2mel.restore(transformed_mel)

    display(Audio(wav, rate=sample_rate))
    display(Audio(transformed_wav, rate=sample_rate))
    plot_transformed_mels(mel, transformed_mel)
    plt.show()

In [ ]:
testing_pairs = [
    (PitchUp, {"num_mels_up": 5}),
    (PitchUp, {"num_mels_up": 10}),
    (PitchDown, {"num_mels_down": 5}),
    (PitchDown, {"num_mels_down": 10}),
    (SpeedUpDown, {"speed_up_factor": 0.5}),
    (SpeedUpDown, {"speed_up_factor": 0.8}),
    (SpeedUpDown, {"speed_up_factor": 1.2}),
    (SpeedUpDown, {"speed_up_factor": 2.0}),
    (Loudness, {"loudness_factor": 0.5}),
    (Loudness, {"loudness_factor": 2.0}),
    (TimeReverse, {}),
    (VerticalSwap, {}),
    (WeakFrequenciesRemoval, {"quantile": 0.9}),
    (WeakFrequenciesRemoval, {"quantile": 0.99}),
    (Cringe1, {}),
    (Cringe2, {}),
]
for test_class, kwargs in testing_pairs:
    print(f"{test_class.__name__}")
    try:
        show_transformation(test_class, wav, kwargs)
    except NotImplementedError as e:
        print(f"Class {test_class.__name__} is not implemented yet")